In [1]:
from matplotlib import pyplot
import numpy as np
from helmi import Helmi

h = Helmi(10000)
h.run_circuits_on_Adonis()
h.run_circuits_on_Aer()
h.pickle_results()

if False:
    for i, (x1, x2) in enumerate(zip(expec1, expec2)):
        print(np.abs(x1, x2))
        
        if i < 10:
            fig = circuits[i].draw(output="mpl")
            display(fig)
            pyplot.close(fig)


c:\Users\valte\OneDrive - University of Helsinki\Desktop\nanoGPT-for-error-mitigation\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Native operations of the backend: ['measure', 'id', 'r', 'cz', 'reset']
Coupling map of the backend: [[0, 2], [2, 0], [1, 2], [2, 1], [3, 2], [2, 3], [4, 2], [2, 4]]
Qubit list of the backend: 5


In [15]:
import rustworkx as rx
import networkx as nx

def dag_node_to_dict(dag_node):
    return {
        'successors': dag_node.successors if dag_node.successors is not None else [],
        'predecessors': dag_node.predecessors if dag_node.predecessors is not None else [],
        'matchedwith': dag_node.matchedwith if dag_node.matchedwith is not None else [],
        'successorstovisit': dag_node.successorstovisit if dag_node.successorstovisit is not None else [],
        'isblocked': dag_node.isblocked if dag_node.isblocked is not None else False,
        'qindices': dag_node.qindices if dag_node.qindices is not None else [],
        'cindices': dag_node.cindices if dag_node.cindices is not None else []
    }

def rustworkx_to_networkx(rx_graph: rx.PyDiGraph) -> nx.DiGraph:
    # Create a new directed graph
    nx_graph = nx.DiGraph()

    # Add nodes to the NetworkX graph
    for node in rx_graph.node_indices():
        dag_node = rx_graph[node]
        nx_graph.add_node(node, feature = dag_node_to_dict(dag_node))

    # Add edges to the NetworkX graph
    for edge in rx_graph.edge_list():
        edge_data = rx_graph.get_edge_data(edge[0], edge[1])
        nx_graph.add_edge(edge[0], edge[1], **edge_data)

    return nx_graph

In [17]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_dagdependency
from karateclub.graph_embedding import Graph2Vec # type: ignore
from rustworkx.visualization import mpl_draw

# Create a Quantum Circuit acting on the q register
circ = QuantumCircuit(2, 2)
circ.h(0)
circ.cx(0, 1)
circ.measure_all()

dag = circuit_to_dagdependency(circ)
rxgraph = dag.to_retworkx()
nxgraph = rustworkx_to_networkx(rxgraph)

# The default values except attributed=True
vectorizer = Graph2Vec(wl_iterations = 2, 
                       attributed = True, 
                       dimensions = 128, 
                       workers = 8, 
                       down_sampling = 0.0001, 
                       epochs = 10, 
                       learning_rate = 0.025, 
                       min_count = 1, 
                       seed = 42, 
                       erase_base_features = False)
vectorizer.fit([nxgraph])
vectors = vectorizer.get_embedding()
print(vectors)



[[ 6.2457966e-03  7.3214006e-03  6.2413486e-03 -5.9920042e-03
  -6.6339737e-03 -3.5399797e-03 -3.3481230e-03  3.5996926e-03
  -5.2103796e-03 -6.6105318e-03 -3.6146259e-03  7.5923381e-03
  -3.6424783e-03  1.1722541e-03  6.8748463e-03  3.3873690e-03
  -7.5965701e-04  7.2954777e-03 -4.6499781e-03  4.3219235e-03
   3.1530487e-03 -2.7514528e-03 -7.2960241e-04 -6.8855383e-03
   4.1216274e-05 -2.0750505e-03  7.7505037e-03 -3.6903576e-03
  -5.2833459e-03  8.7821909e-04 -6.1130440e-03 -6.2287301e-03
  -5.1264050e-03 -4.7671548e-03 -3.4000087e-03 -1.5353976e-03
  -1.9415796e-03  4.5094336e-03  5.3437473e-03 -3.5461499e-03
  -4.9601458e-03  6.1006908e-04 -3.5258564e-03  2.0201718e-05
   5.6116018e-03  3.0373819e-03 -4.2307796e-03 -1.0032243e-03
   1.4305540e-04 -6.3201259e-03  7.7113803e-03  6.5023359e-03
  -2.3321647e-05  4.5744120e-03  3.5238252e-03  6.7724125e-03
   2.8544490e-03 -7.2570248e-03 -1.9170014e-03  2.8053687e-03
  -4.4786994e-04  3.8329775e-03  2.7570508e-03  3.4603688e-03
   4.334